In [44]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [45]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()


In [46]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.option("header", "true").csv(SparkFiles.get("home_sales_revised.csv"), inferSchema = True, sep = ",")
df.show()


23/05/28 01:53:25 WARN SparkContext: The path https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv has been added already. Overwriting of added paths is not supported in the current version.
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b

In [50]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")


In [51]:
result = spark.sql("""
    SELECT EXTRACT(YEAR FROM date) AS year, ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year
    ORDER BY year
""")

result.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [54]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
result = spark.sql("""
    select extract (year from date) as year, round(avg(price), 2) as `average price of a home`
    from home_sales
    where bedrooms == 3 and bathrooms == 3
    group by year
    """)
result.show()

+----+-----------------------+
|year|average price of a home|
+----+-----------------------+
|2022|              292725.69|
|2019|              287287.82|
|2020|              294204.16|
|2021|              294211.46|
+----+-----------------------+



In [55]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
result = spark.sql("""
    select extract (year from date) as year, round(avg(price), 2) as `average price of a home`
    from home_sales
    where bedrooms == 3 and bathrooms == 3 and sqft_living >= 2000
    group by year
    """)
result.show()

+----+-----------------------+
|year|average price of a home|
+----+-----------------------+
|2022|              302871.28|
|2019|              294372.02|
|2020|               296314.6|
|2021|              305118.97|
+----+-----------------------+



In [56]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

result = spark.sql("""
    select round(avg(price), 2) as `average price of a home`
    from home_sales
    where price >= 350000
    group by view
    """)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+-----------------------+
|average price of a home|
+-----------------------+
|              399856.95|
|             1056336.74|
|              736679.93|
|               755214.8|
|             1080649.37|
|              401419.75|
|             1053472.79|
|              402124.62|
|             1058802.78|
|              401506.97|
|              399537.66|
|              400598.05|
|              401501.32|
|             1137372.73|
|              402022.68|
|             1026006.06|
|               398447.5|
|              401044.25|
|              733780.26|
|              398917.98|
+-----------------------+
only showing top 20 rows

--- 0.13721394538879395 seconds ---


In [57]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [58]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [59]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

result = spark.sql("""
    select view, round(avg(price), 2) as `ratings with average price`
    from home_sales
    where price >= 350000
    group by view
    """)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------------+
|view|ratings with average price|
+----+--------------------------+
|  31|                 399856.95|
|  85|                1056336.74|
|  65|                 736679.93|
|  53|                  755214.8|
|  78|                1080649.37|
|  34|                 401419.75|
|  81|                1053472.79|
|  28|                 402124.62|
|  76|                1058802.78|
|  26|                 401506.97|
|  27|                 399537.66|
|  44|                 400598.05|
|  12|                 401501.32|
|  91|                1137372.73|
|  22|                 402022.68|
|  93|                1026006.06|
|  47|                  398447.5|
|   1|                 401044.25|
|  52|                 733780.26|
|  13|                 398917.98|
+----+--------------------------+
only showing top 20 rows

--- 0.10980510711669922 seconds ---


In [60]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partition")

In [61]:
# 11. Read the formatted parquet data.
parquet_data = spark.read.parquet("home_sales_partition")

In [62]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView("home_sales_partition")

In [64]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

result = spark.sql("""
    select view, round(avg(price), 2) as `ratings with average price`
    from home_sales_partition
    where price >= 350000
    group by view
    """)
result.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------------+
|view|ratings with average price|
+----+--------------------------+
|  31|                 399856.95|
|  85|                1056336.74|
|  65|                 736679.93|
|  53|                  755214.8|
|  78|                1080649.37|
|  34|                 401419.75|
|  81|                1053472.79|
|  28|                 402124.62|
|  76|                1058802.78|
|  26|                 401506.97|
|  27|                 399537.66|
|  44|                 400598.05|
|  12|                 401501.32|
|  91|                1137372.73|
|  22|                 402022.68|
|  93|                1026006.06|
|  47|                  398447.5|
|   1|                 401044.25|
|  52|                 733780.26|
|  13|                 398917.98|
+----+--------------------------+
only showing top 20 rows

--- 0.21045994758605957 seconds ---


In [65]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [66]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")

False